In [1]:
!pip install pandas requests pyarrow fastparquet tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 20.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 18.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.0/687.0 kB 15.3 MB/s eta 0:00:00a 0:00:01
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 14.7 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.1 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [3]:
import pandas as pd
import os
import requests
from tqdm import tqdm

# Настройки
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
year = 2022
months = [1]  # Можно указать несколько месяцев
local_dir = "nyc_taxi_data"
os.makedirs(local_dir, exist_ok=True)

dfs = []

for month in tqdm(months):
    fname = f"yellow_tripdata_{year}-{month:02d}.parquet"
    url = base_url + fname
    local_path = os.path.join(local_dir, fname)

    # Скачиваем файл, если его ещё нет
    if not os.path.exists(local_path):
        print(f"Скачиваем {fname}...")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
        else:
            print(f"Ошибка загрузки {url}: HTTP {response.status_code}")
            continue

    # Загружаем в DataFrame
    df = pd.read_parquet(local_path)
    dfs.append(df)

# Объединяем
if dfs:
    taxi_df = pd.concat(dfs)
    print(f"Загружено строк: {len(taxi_df):,}")
    display(taxi_df.head())
else:
    print("Не удалось загрузить ни один файл.")

  0%|          | 0/1 [00:00<?, ?it/s]

Скачиваем yellow_tripdata_2022-01.parquet...


100%|██████████| 1/1 [00:04<00:00,  4.30s/it]

Загружено строк: 2,463,931


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
import pandas as pd
import os
import requests
from tqdm import tqdm

def download_nyc_taxi_data(year: int, month: int, types=None):
    if types is None:
        types = ['yellow', 'green', 'fhv', 'fhvhv']
    
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
    local_dir = "nyc_taxi_all"
    os.makedirs(local_dir, exist_ok=True)
    dfs = []

    for t in types:
        fname = f"{t}_tripdata_{year}-{month:02d}.parquet"
        url = base_url + fname
        local_path = os.path.join(local_dir, fname)

        # Скачиваем
        if not os.path.exists(local_path):
            print(f"Скачиваем {fname}...")
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                with open(local_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
            else:
                print(f"[!] Пропущен {t}: HTTP {response.status_code}")
                continue

        # Загружаем
        try:
            df = pd.read_parquet(local_path)
            df['source_type'] = t  # Добавляем метку источника
            dfs.append(df)
        except Exception as e:
            print(f"[!] Ошибка чтения {fname}: {e}")

    if dfs:
        all_df = pd.concat(dfs, ignore_index=True)
        print(f"Всего строк загружено: {len(all_df):,}")
        return all_df
    else:
        print("Данные не загружены.")
        return pd.DataFrame()

In [6]:
df_all = download_nyc_taxi_data(2022, 1)
df_all.head()

Скачиваем yellow_tripdata_2022-01.parquet...
Скачиваем green_tripdata_2022-01.parquet...
Скачиваем fhv_tripdata_2022-01.parquet...
Скачиваем fhvhv_tripdata_2022-01.parquet...
Всего строк загружено: 18,421,708


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls,bcf,sales_tax,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,1.0,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142.0,236.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236.0,42.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166.0,166.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114.0,68.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68.0,163.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18421708 entries, 0 to 18421707
Data columns (total 49 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   VendorID                float64       
 1   tpep_pickup_datetime    datetime64[us]
 2   tpep_dropoff_datetime   datetime64[us]
 3   passenger_count         float64       
 4   trip_distance           float64       
 5   RatecodeID              float64       
 6   store_and_fwd_flag      object        
 7   PULocationID            float64       
 8   DOLocationID            float64       
 9   payment_type            float64       
 10  fare_amount             float64       
 11  extra                   float64       
 12  mta_tax                 float64       
 13  tip_amount              float64       
 14  tolls_amount            float64       
 15  improvement_surcharge   float64       
 16  total_amount            float64       
 17  congestion_surcharge    float64       
 18  

In [ ]:
df_all_2023 = download_nyc_taxi_data(2023, 1)
df_all_2023.head()

Скачиваем yellow_tripdata_2023-01.parquet...
Скачиваем green_tripdata_2023-01.parquet...
Скачиваем fhv_tripdata_2023-01.parquet...
Скачиваем fhvhv_tripdata_2023-01.parquet...
Всего строк загружено: 22,728,328


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls,bcf,sales_tax,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,2.0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161.0,141.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43.0,237.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48.0,238.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138.0,7.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107.0,79.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/aleksandrbogachenkov/.pyenv/versions/3.10.13/lib/python3.10/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/Users/aleksandrbogachenkov/.pyenv/versions/3.10.13/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 339, in dispatch_control
    await self.process_control(msg)
  File "/Users/aleksandrbogachenkov/.pyenv/versions/3.10.13/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 345, in process_control
    idents, msg = self.session.feed_identities(msg, copy=False)
  File "/Users/aleksandrbogachenkov/.pyenv/versions/3.10.13/lib/python3.10/site-packages/jupyter_client/session.py", line 994, in feed_identities
    raise ValueError(msg)
ValueError: DELIM not in msg_list
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/aleksandrbogachenkov/.pyenv/versions/

In [9]:
df_all_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22728328 entries, 0 to 22728327
Data columns (total 49 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   VendorID                float64       
 1   tpep_pickup_datetime    datetime64[us]
 2   tpep_dropoff_datetime   datetime64[us]
 3   passenger_count         float64       
 4   trip_distance           float64       
 5   RatecodeID              float64       
 6   store_and_fwd_flag      object        
 7   PULocationID            float64       
 8   DOLocationID            float64       
 9   payment_type            float64       
 10  fare_amount             float64       
 11  extra                   float64       
 12  mta_tax                 float64       
 13  tip_amount              float64       
 14  tolls_amount            float64       
 15  improvement_surcharge   float64       
 16  total_amount            float64       
 17  congestion_surcharge    float64       
 18  

In [1]:
!pip install kaggle


  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [kaggle]2m5/6 [kaggle]


In [ ]:

# Указать нужный датасет (историческая погода)
!kaggle datasets download -d selfishgene/historical-hourly-weather-data -p weather_data --unzip